# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.get_data import *
from IPython.display import clear_output
from sklearn.linear_model import Ridge

# Load data

In [3]:
%load_ext autoreload
%autoreload 2

b_data, u_data, reviews = get_training_data(verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
========== Feature 'attributes_GoodForKids' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_OutdoorSeating' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsDelivery' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsGoodForGroups' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsPriceRange2' ==========
TYPE: numeric.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsReservations' ==========
TYPE: boolea

========== Feature 'attributes_GoodForKids' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_OutdoorSeating' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsDelivery' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsGoodForGroups' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsPriceRange2' ==========
TYPE: numeric.
Detected NaN in column. Replacing with mean of non-NaN values.

========== Feature 'attributes_RestaurantsReservations' ==========
TYPE: boolean. Changing False -> 0, True -> 1.
Detected NaN in column. Replacing with mean of non-Na

# Verify data was cleaned correctly

In [38]:
for col in b_data.columns:
    u_vals = b_data[col].unique()
    print(col, u_vals)
    input()
    clear_output()
    
b_data.head()

,attributes_GoodForKids,attributes_OutdoorSeating,attributes_RestaurantsDelivery,attributes_RestaurantsGoodForGroups,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_WiFi,stars
business_id,,,,,,,,
KuxDPl6UYNLxFChPm0_MNw,0.0,1.0,0.0,1.0,2.0,1.0,0,4.0
6SAfQKe2oM5g_EtcYXyAMg,1.0,1.0,0.0,1.0,1.0,0.0,0,4.0
upB0RQl-l529IVwgOpwOQQ,0.0,1.0,0.0,1.0,1.0,0.0,0,4.5
TulmRC5V0--dnXYd_GOSvA,1.0,0.0,0.0,1.0,2.0,0.0,0,4.0
yqYtY3-Po4OVPafA9Z-Xyw,1.0,0.0,0.0,1.0,2.0,1.0,1,3.5


# Regression

## Create design matrix for training data

In [4]:
N = len(reviews['stars'])
Db = len(b_data.columns)
Du = len(u_data.columns)
D = Db + Du

X = np.zeros((N, D))
y = np.zeros(N)

for i, review in reviews.iterrows():
    if (i % 20000) == 0:
        print('%d/%d done' % (i, N))
    
    u_id = review['user_id']
    b_id = review['business_id']
    y[i] = review['stars']
    
    X[i, :Db] = b_data.loc[b_id].values
    X[i, Db:] = u_data.loc[u_id].values
    
print('Done')

20000/150232 done
40000/150232 done
60000/150232 done
80000/150232 done
100000/150232 done
120000/150232 done
140000/150232 done
Done


## Create design matrix for validation data

In [11]:
v_queries = get_validation_reviews()
Nv = len(v_queries['stars'])
X_valid = np.zeros((Nv, D))
y_valid = np.zeros(Nv)

for i, review in v_queries.iterrows():
    if (i % 20000) == 0:
        print('%d/%d done' % (i, Nv))
        
    u_id = review['user_id']
    b_id = review['business_id']
    y_valid[i] = review['stars']
    
    X_valid[i, :Db] = b_data.loc[b_id].values
    X_valid[i, Db:] = u_data.loc[u_id].values
    
print('Done!')

0/50077 done
20000/50077 done
40000/50077 done
float64
Done!


## Do regression

In [7]:
results = dict()

def mse(y1, y2):
    return np.mean((y1-y2)**2)

In [8]:
clf = Ridge(alpha=0.1)
clf.fit(X, y)

# training error
y_pred_train = clf.predict(X)
mse_train = mse(y, y_pred_train)
print('Training MSE: %f' % mse_train)

y_pred_valid = clf.predict(X_valid)
mse_valid = mse(y_valid, y_pred_valid)
print('Validation MSE: %f' % mse_valid)

Training MSE: 1.012298
Validation MSE: 1.107752


In [10]:
reviews['stars'].unique()

array([4., 5., 2., 3., 1.])